In [6]:
import io
import subprocess
import pickle
import copy
import gzip
import tqdm
import re
from itertools import islice

In [7]:
with gzip.open('theorum.pickle.gzip','rb') as f:
    theorem_gzip = pickle.load(f)


# Length of rows and column
Getting a sense of .gzip data

In [8]:
print("Rows:", len(theorem_gzip))
print("Columns:", len(theorem_gzip[0]))
print(theorem_gzip[0][0])
print(theorem_gzip[0][1])
print(theorem_gzip[0][2])
print(theorem_gzip[0][3])

Rows: 493140
Columns: 4
(a).=>(a).
fof('(a)',axiom,(a)). fof('(a)',conjecture,(a)).
Found
CompletedProcess(args=['/home/anmarch/source/eprover/PROVER/eprover', '--proof-object', 'f.tptp'], returncode=0, stdout=b"# Initializing proof state\n# Scanning for AC axioms\n#\n#cnf(i_0_1, plain, (a)).\n#\n# Proof found!\n# SZS status Theorem\n# SZS output start CNFRefutation\nfof('(a)', conjecture, a, file('f.tptp', '(a)')).\nfof('(a)', axiom, a, file('f.tptp', '(a)')).\nfof(c_0_2, negated_conjecture, ~a, inference(fof_simplification,[status(thm)],[inference(assume_negation,[status(cth)],['(a)'])])).\nfof(c_0_3, negated_conjecture, ~a, inference(fof_nnf,[status(thm)],[c_0_2])).\ncnf(c_0_4, negated_conjecture, (~a), inference(split_conjunct,[status(thm)],[c_0_3])).\ncnf(c_0_5, plain, (a), inference(split_conjunct,[status(thm)],['(a)'])).\ncnf(c_0_6, negated_conjecture, ($false), inference(cn,[status(thm)],[inference(rw,[status(thm)],[c_0_4, c_0_5])]), ['proof']).\n# SZS output end CNFRefutation\

# Cleanup Prep

Write gzip to .txt file

In [9]:
f = open("theorem.txt", "w")
#file.write(output)

#Will later use len(theorem_gzip) instead of what is hardcoded
for x in range(5):
    for y in range(len(theorem_gzip[x])):      
        if x == 0 and y == 0: #for the first line.
            f.write("theorem #" + str(x) + "\n" + str(theorem_gzip[x][y]).replace("\\n", ""))
        elif x != 0 and y == 0: #for the rest of the lines.
            f.write("\ntheorem #" + str(x) + "\n" + str(theorem_gzip[x][y]).replace("\\n", ""))
        elif y == 1: #Adds newline to string because the theorum[x][1] doesn't have newlines.
            f.write("\n" + str(theorem_gzip[x][y]).replace(" ", "\n"))
        else: #Case to consider for other columns.
            f.write("\n" + str(theorem_gzip[x][y]).replace("\\n", "\n"))
f.close()

Reading .txt file

In [10]:
lines = []

def fn_read_file(file):
    with open(file, "r") as f:
        return f.readlines()

lines = fn_read_file("theorem.txt")

Deleting lines we don't need. Examples:
* Lines containing .tptp
* CompletedProces (args=['/home/user/...

In [11]:
with open("theorem.txt", "w") as f:
    for line in lines:
        #Problem: not able to remove the last string. 
        if ("tptp" not in line) and ("#\n" != line) and ("\", stderr=b\'\')\n" != line):
            f.write(line)
lines = fn_read_file("theorem.txt")

Grab axioms and conjectures

In [12]:
axioms = []
conjects = []
with open("test.txt", "r") as f:
    for line in lines: 
        #Grabs axioms
        if ("fof(\'(" in line) and ("conjecture" not in line):
            temp_axiom = ""  
            for char in line[6:]:
                if char == ")":  # Stop collecting if reached closed parentheses
                    break  
                temp_axiom += char  # Append to temp_axiom
            axioms.append(temp_axiom)  # Append full axiom 
        
        #Grabs conjectures
        if ("fof(\'(" in line) and ("conjecture" in line):
            temp_conject = ""
            find_conjecture = re.search(r'\b' + re.escape("conjecture") + r'\b', line)  # \b is for word boundaries
            for char in line[(find_conjecture.end() + 2):]:
                if char == ")":
                    break
                temp_conject += char
            conjects.append(temp_conject)